In [ ]:
import pyomo.environ as pyo
import idaes
from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock)
import idaes.core.util.scaling as iscale
from ammonia_PR import get_prop
import pytest
from SofcDesignCopy import Nh3SofcDesign, EosType

In [ ]:

def sofc_flowsheet(eos=EosType.PR):
    m = pyo.ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    sweep_comp = {
        "O2":0.2074,
        "H2O":0.0099,
        "CO2":0.0003,
        "N2":0.7732,
        "Ar":0.0092,
    }

    feed_comp = {
        "NH3": 0.15,  # 15% ammonia fuel
        "H2O": 0.10,  # 10% steam for thermal balance, help shift reactions
        "N2": 0.70,   # Balance gas or carrier (from cracked NH₃ or diluent)
        "O2": 0.05    # Only if pre-mixed air (usually not at anode, but cathode)
    }


    m.fs.air_side_prop_params = GenericParameterBlock(
        **get_prop(sweep_comp, {"Vap"}, eos=eos),
        doc="Air property parameters",
    )
    m.fs.flue_side_prop_params = GenericParameterBlock(
        **get_prop(feed_comp, {"Vap"}, eos=eos),
        doc="Flue gas property parameters",
    )
    m.fs.sofc = Nh3SofcDesign(
        air_side_property_package=m.fs.air_side_prop_params,
        fuel_side_property_package=m.fs.fuel_side_prop_params,
        reaction_eos=eos
    )

    m.fs.sofc.fuel_side_inlet.temperature.fix(1023) #change
    m.fs.sofc.fuel_side_inlet.pressure.fix(20e5) #change
    m.fs.sofc.fuel_side_inlet.flow_mol.fix(2) #change
    for (t, i), c in m.fs.sofc.fuel_side_inlet.mole_frac_comp.items():
        c.fix(feed_comp[i])
    m.fs.sofc.air_side_inlet.temperature.fix(1023)
    m.fs.sofc.air_side_inlet.pressure.fix(20e5)
    m.fs.sofc.air_side_inlet.flow_mol.fix(2)
    for (t, i), c in m.fs.sofc.air_side_inlet.mole_frac_comp.items():
        c.fix(sweep_comp[i])
    m.fs.sofc.fuel_side_outlet_temperature.fix(1023)
    m.fs.sofc.air_side_outlet_temperature.fix(1023)
    m.fs.sofc.fuel_utilization.fix(0.7)
    iscale.calculate_scaling_factors(m)
    m.fs.sofc.initialize(optarg={"max_iter":30})
    return m

# create a flowsheet with the SOFC model
model = sofc_flowsheet()

In [ ]:
import idaes_ui
import idaes_ui.fv
dir(idaes_ui.fv)
from idaes_ui.fv import visualize 

In [ ]:
# Visualize the model
visualize(model.fs, "SOFC")